In [1]:
from utils_1551 import Corpus, load_train_and_test

from bpemb import BPEmb
from uk_stopwords import STOP_WORDS
from typing import List
import numpy as np

In [2]:
train, test = load_train_and_test()

In [3]:
dim = 300
bpemb_ua = BPEmb(lang="uk", dim=dim)

In [4]:
print(bpemb_ua.embed('щось там').shape)
print(bpemb_ua.encode('щось там'))

(3, 300)
['▁що', 'сь', '▁там']


In [5]:
import re
re_tokens = re.compile("([\w][\w]*'?\w?)")

def text2tokens(text: str)->List[str]:
    return re_tokens.findall(text)

s = "Ваша відповідь:  Повідомляємо, що відповідно до п. 5.7 Державного стандарту України 4123-2006 «Пристрій примусового зниження швидкості дорожньо-транспортної техніки на вулицях і дорогах. Загальні технічні вимоги» заборонено встановлювати пристрої примусового зниження швидкості руху транспортних засобів на магістральних вулицях, а просп. Григоренка має категорію магістральної вулиці районного значення."
print(text2tokens(s))

['Ваша', 'відповідь', 'Повідомляємо', 'що', 'відповідно', 'до', 'п', '5', '7', 'Державного', 'стандарту', 'України', '4123', '2006', 'Пристрій', 'примусового', 'зниження', 'швидкості', 'дорожньо', 'транспортної', 'техніки', 'на', 'вулицях', 'і', 'дорогах', 'Загальні', 'технічні', 'вимоги', 'заборонено', 'встановлювати', 'пристрої', 'примусового', 'зниження', 'швидкості', 'руху', 'транспортних', 'засобів', 'на', 'магістральних', 'вулицях', 'а', 'просп', 'Григоренка', 'має', 'категорію', 'магістральної', 'вулиці', 'районного', 'значення']


In [6]:
def text2vec(text: str):
    res = np.zeros(dim)
    words = text2tokens(text.lower())
    for w in words:
        if w in STOP_WORDS:
            continue
        for v in bpemb_ua.embed(w):
            res += v
    res /= max(np.linalg.norm(res), 1e-6)
    return res

print(text2vec(s).shape)

(300,)


In [7]:
def corpus2xy(corpus: Corpus)->(np.array, np.array):
    x, y = [], []
    for name, messages in corpus.items():
        for id_, text in messages:
            vec = text2vec(text)
            x.append(vec)
            y.append(name)
    return np.array(x), np.array(y)


In [8]:
x_train, y_train = corpus2xy(train)

In [9]:
print(x_train.shape, x_train.dtype)
print(y_train.shape, y_train.dtype)

(48120, 300) float64
(48120,) <U80


In [10]:
x_test, y_test = corpus2xy(test)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=20)
clf.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=20, p=2,
           weights='uniform')

In [12]:
y_pred = clf.predict(x_test)

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

C:\Users\ssotn\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                                                                  precision    recall  f1-score   support

                                                                     Інші-Подяки       0.20      0.03      0.05        78
                                              Інші-технічні-недоліки-стану-ліфту       0.24      0.55      0.33       182
                Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.02      0.01      0.02        73
                                          Бажаючі-отримати--Картки-киянина--КК--       0.32      0.42      0.36        19
                              Будівництво--дооблаштування-спортивних-майданчиків       0.33      0.09      0.14        22
                                                                 Будівництво-АЗС       0.38      0.12      0.18        25
                                                        Будівництво-в-нічний-час       1.00      0.07      0.12        15
                       

In [14]:
clf10 = KNeighborsClassifier(n_neighbors=10)
clf10.fit(x_train, y_train)
y_pred10 = clf10.predict(x_test)
print(classification_report(y_test, y_pred10))

C:\Users\ssotn\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                                                                  precision    recall  f1-score   support

                                                                     Інші-Подяки       0.16      0.08      0.10        78
                                              Інші-технічні-недоліки-стану-ліфту       0.20      0.51      0.29       182
                Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.04      0.05      0.05        73
                                          Бажаючі-отримати--Картки-киянина--КК--       0.21      0.37      0.26        19
                              Будівництво--дооблаштування-спортивних-майданчиків       0.29      0.18      0.22        22
                                                                 Будівництво-АЗС       0.22      0.20      0.21        25
                                                        Будівництво-в-нічний-час       0.22      0.27      0.24        15
                       

In [15]:
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(x_train, y_train)
y_pred2 = clf2.predict(x_test)
print(classification_report(y_test, y_pred2))

                                                                                  precision    recall  f1-score   support

                                                                     Інші-Подяки       0.21      0.14      0.17        78
                                              Інші-технічні-недоліки-стану-ліфту       0.24      0.58      0.34       182
                Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою       0.15      0.18      0.17        73
                                          Бажаючі-отримати--Картки-киянина--КК--       0.45      0.53      0.49        19
                              Будівництво--дооблаштування-спортивних-майданчиків       0.32      0.36      0.34        22
                                                                 Будівництво-АЗС       0.28      0.36      0.32        25
                                                        Будівництво-в-нічний-час       0.21      0.40      0.28        15
                       